# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других.

**Цель** — необходимо сделать модель, которая будет искать токсичные комментарии и отправлять их на модерацию.

**Задача** — Обучить модель классифицировать комментарии на позитивные и негативные со значением метрики качества *F1* не меньше 0.75.

**План**

1. Изучение и подготовка данных.
2. Обучение моделей
3. Вывод

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Изучение и подготовка данных
### Импорт библиотек

In [16]:
import os
import warnings
import re

import pandas as pd
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import (train_test_split,
                                     cross_val_score, RandomizedSearchCV)
from sklearn.metrics import f1_score
from sklearn.exceptions import ConvergenceWarning
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from tqdm.notebook import tqdm

import transformers
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from lightgbm import LGBMClassifier

In [2]:
pd.options.mode.chained_assignment = None
warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', UserWarning)

In [3]:
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\stepa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\stepa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stepa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stepa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\stepa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Функции

In [59]:
def lemmatize(text):
    '''
    Функция делает токенизацию, а затем лемматизацию слов и
    возвращает список лемматизированных предложений
    '''

    # Инициализируем класс для лемматизации
    lemmatizer = WordNetLemmatizer()
    lt = []
    for sentence in text:
        # Вызываем функцию для отчистки текста
        sentence_clear = clear_text(sentence)
        tokens = word_tokenize(sentence_clear)  # Делаем токенизацию
        # Делаем лемматизацию
        lemmatized_sentence = [lemmatizer.lemmatize(
            token, get_wordnet_pos(token)) for token in tokens]
        # Объединяем слова в предложение
        lemm_text = ' '.join(lemmatized_sentence)
        lt.append(lemm_text)

    return lt

In [60]:
def get_wordnet_pos(word):
    '''
    Сопоставьте тег POS с первым символом lemmatize()
    '''
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [61]:
def clear_text(text):
    '''
    Функция очищает текст от лишних симолов
    '''
    pattern = r'[^a-zA-Z]'
    re_text = re.sub(pattern, ' ', text)
    total_text = ' '.join(re_text.split()).lower()
    return total_text

### Константы

In [4]:
PTH = r"C:\Users\stepa\Downloads\toxic_comments.csv"
RANDOM_STATE = 42
TEST_SIZE = 0.25

### Загрузка данных

Прочитаем файл `toxic_comments.csv` и сохраним его в `df_toxic`

In [5]:
if os.path.exists(PTH):
    df_toxic = pd.read_csv(PTH, index_col=0)
else:
    print('Ошибка, файл не найден')

Посмотрим на первые 5 строк датафрейма

In [ ]:
df_toxic.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Посмотрим на размерность датафрейма

In [ ]:
df_toxic.shape

(159292, 2)

Теперь посмотрим на балан классов в таргете.

In [ ]:
df_toxic['toxic'].value_counts()

,count
toxic,
0,143106
1,16186


Получаем сильный дисбаланс классов.

### Подготовка данных

Приступим к подготовке данных. Для этого создадим корпусов текстов. Затем проведем лемматизацию, используя модуль `nltk`, то есть приведем слова к начальной форме, после этого отчистим текст от лишних знаков, чтобы остались только буквы. Удалим стоп слова и посчитаем оценку важности слов (`TF-IDF`)

In [62]:
# Создаем корпус текстов
corpus = df_toxic['text'].values

In [63]:
lemm_text = lemmatize(corpus)

Добавим лемматизированные предложения в датафрейм, как новый признак.

In [64]:
# Добавляем лемматизированые предложения в датафрейм
df_toxic['lemm_text'] = lemm_text

In [65]:
# Выведем первые 5 строк
df_toxic.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [66]:
# Удалим корпус, для экономии памяти
del corpus

Перед тем как начать считать `TF-IDF` разделим данные на тренировчную и тестовую выборки

In [67]:
# Отделяем таргет
X = df_toxic.drop(columns='toxic', axis=1)
y = df_toxic['toxic']

In [68]:
# Разделяем данные на выборки
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=RANDOM_STATE,
    test_size=TEST_SIZE,
    stratify=y
)

Теперь создадим корпуст по лемматизированым текстам

In [69]:
# Создаем корпус текстов
corpus = X_train['lemm_text'].values

In [70]:
# Создаем корпус текстов
corpus_test = X_test['lemm_text'].values

In [71]:
# Загузим английские стоп-слова
stopwords = list(nltk_stopwords.words('english'))

Оценку важности слов (`TF-IDF`) будем проводить в пайплайне вместе с обучением модели при использовании кросс-валидации.

In [72]:
# Создаем пайплайн, в котором объединяем векторизатор и модель
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords)),
        ("model", LogisticRegression(random_state=RANDOM_STATE)),
    ]
)

Можем приступать к обучению моделей

## Обучение моделей

Перед нами стоит задача классификации. Целевой признак `toxic`. Будем обучать 4 модели с использованием кросс-валидации:
- `LogisticRegression`
- `DecisionTreeClassifier`
- `SVM`
- `LGBMClassifier`

Проверять результаты моделей будем на метрике `F1`, она должна быть больше 75. После этого выберем лучшую модель и подберем ей гиперпараметры методом `RandomizedSearchCV`.

### Базовые модели

#### LogisticRegression

Начнем с логистической регрессии с кросс-валидацией.

In [ ]:
# Выполняем кросс-валидацию
score_lg = cross_val_score(pipeline, corpus, y_train, cv=5, scoring='f1')

In [ ]:
print(f'Результат на кросс-валидации: {score_lg.mean():.2f}')
print(f'Отклонение на кросс-валидации: {score_lg.std():.2%}')

Результат на кросс-валидации: 0.72
Отклонение на кросс-валидации: 0.57%


Получаем метрику на кросс-валидации 0.72 и отклонение 0.57%.

#### DecisionTreeClassifier

Теперь приступим к дереву решения. Для кросс-валидации будем использовать метод `cross_val_score`.

In [ ]:
# Обновляем модель в существующем пайплайне на DecisionTreeClassifier
pipeline.set_params(model=DecisionTreeClassifier(random_state=RANDOM_STATE))

Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('model', DecisionTreeClassifier(random_state=42))])

In [ ]:
# Выполняем кросс-валидацию
score_tree = cross_val_score(pipeline, corpus, y_train, cv=5, scoring='f1')

In [ ]:
print(f'Результат на кросс-валидации: {score_tree.mean():.2f}')
print(f'Отклонение на кросс-валидации: {score_tree.std():.2%}')

Результат на кросс-валидации: 0.71
Отклонение на кросс-валидации: 0.31%


Результат на кросс-валидации 0.71, что хуже, чем у логистической регрессии, но зато отклонение 0.31%.

#### SVM

Предпоследним алгоритмом будет метод опорных векторов с ядром `rbf`

In [ ]:
# Обновляем модель в существующем пайплайне на SVM
pipeline.set_params(model=SVC(random_state=RANDOM_STATE, kernel='rbf'))

Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('model', SVC(random_state=42))])

In [ ]:
# Выполняем кросс-валидацию
score_svc = cross_val_score(pipeline, corpus, y_train, cv=5, scoring='f1')

In [ ]:
print(f'Результат на кросс-валидации: {score_svc.mean():.2f}')
print(f'Отклонение на кросс-валидации: {score_svc.std():.2%}')

Результат на кросс-валидации: 0.74
Отклонение на кросс-валидации: 0.53%


Результат на кросс-валидации 0.74, а также большие отклонения 0.53%. Пока что это лучшая модель

#### LGMBClassifier

Последним попробуем градиентный бустинг из библиотеки LGBM

In [ ]:
# Обновляем модель в существующем пайплайне на LGBMClassifier
pipeline.set_params(model=LGBMClassifier(
    random_state=RANDOM_STATE, n_jobs=-1, objective='binary', verbosity=-1))

Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('model',
                 LGBMClassifier(n_jobs=-1, objective='binary', random_state=42,
                                verbosity=-1))])

In [ ]:
# Выполняем кросс-валидацию
score_lgbm = cross_val_score(pipeline, corpus, y_train, cv=5, scoring='f1')

In [ ]:
print(f'Результат на кросс-валидации: {score_lgbm.mean():.2f}')
print(f'Отклонение на кросс-валидации: {score_lgbm.std():.2%}')

Результат на кросс-валидации: 0.74
Отклонение на кросс-валидации: 0.84%


Получаем 0.74 на кросс-валидации и отклонение 0.84.

Сравним все модели между собой, результат сравнения свели в таблицу

| **Модели**  |  **F1 на кросс-валидации** | **Отклонение метрики**  | **Удовлетворяет условию** |
|:------------- |:---------------:| :-------------:| :-------------:|
| **LogisticRegressionCV**  |<span style="color:red"> 0.72</span> | 0.57 | ❌ |
| **SVM** |<span style="color:red"> 0.74</span>| 0.53 | ❌ |
| **DecisionTreeRegressor**   |<span style="color:red"> 0.71</span>| 0.31 | ❌|
| **LGBMClassifier**   |<span style="color:red"> 0.74</span>| 0.85 | ❌|

По итогу выбор между `SVM` и `LGBMClassifier`. Хоть отклонение у `LGBMClassifier` больше, но она обучалась быстрее, поэтому дальше будем работать с этой моделью.

### LGMBClassifier подбор гиперпараметров

Так ни одна из моделей не удовлетворяет условию задачи, попробуем улучшить результат подобрав гиперпараметры для лучшей базовой модели. 

Параметры этой модели будем подбирать с помощью `RandomizedSearchCV()`:

- `num_leaves`
- `n_estimators`
- `min_child_samples`
- `max_depth`

In [84]:
# Обновляем модель в существующем пайплайне на LGBMClassifier
pipeline.set_params(model=LGBMClassifier(
    random_state=RANDOM_STATE, n_jobs=-1, objective='binary', verbosity=-1))

Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('model',
                 LGBMClassifier(n_jobs=-1, objective='binary', random_state=42,
                                verbosity=-1))])

In [85]:
# Создаём словарь со значениями гиперпараметров для перебора
parameters_lgbm = {
    'model__num_leaves': [50, 100, 130],
    'model__n_estimators': [1000, 1500, 2000],
    'model__max_depth': range(2, 10),
    'model__min_child_samples': [30, 50]
}

In [86]:
# Инициализируем класс для случайного поиска
randomized_search_lgbm = RandomizedSearchCV(
    pipeline,
    parameters_lgbm,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    random_state=RANDOM_STATE
)

In [87]:
randomized_search_lgbm.fit(corpus, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vect',
                                              TfidfVectorizer(stop_words=['i',
                                                                          'me',
                                                                          'my',
                                                                          'myself',
                                                                          'we',
                                                                          'our',
                                                                          'ours',
                                                                          'ourselves',
                                                                          'you',
                                                                          "you're",
                                                                          "you've",
                                                                          "you'll",
                                                                          "you'd",
                                                                          'your',
                                                                          'yours',
                                                                          'yourself',
                                                                          'yourselves',
                                                                          'he',
                                                                          'him',
                                                                          'his',
                                                                          'himself',
                                                                          'she',
                                                                          "she's",
                                                                          'her',
                                                                          'hers',
                                                                          'herself',
                                                                          'it',
                                                                          "it's",
                                                                          'its',
                                                                          'itself', ...])),
                                             ('model',
                                              LGBMClassifier(n_jobs=-1,
                                                             objective='binary',
                                                             random_state=42,
                                                             verbosity=-1))]),
                   n_jobs=-1,
                   param_distributions={'model__max_depth': range(2, 10),
                                        'model__min_child_samples': [30, 50],
                                        'model__n_estimators': [1000, 1500,
                                                                2000],
                                        'model__num_leaves': [50, 100, 130]},
                   random_state=42, scoring='f1')

In [100]:
print('Лучшая модель и её параметры:\n\n',
      randomized_search_lgbm.best_estimator_)

print(f'Метрика F1 на кросс-валидации: \
{randomized_search_lgbm.best_score_:.2f}')

print(f'Отклонение метрики на кросс-валидации: \
{pd.DataFrame(randomized_search_lgbm.cv_results_)["std_test_score"].mean():.2f}')

Лучшая модель и её параметры:

 Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('model',
                 LGBMClassifier(max_depth=7, min_child_samples=30,
                                n_estimators=2000, n_jobs=-1, num_leaves=100,
                                objective='binary', random_state=42,
                                verbosity=-1))])
Метрика F1 на кросс-валидации: 0

Получили метрику F1 - 0.77, что больше 0.75, а значит данная модель удовлетворяет условию задачи. Теперь можем делать предсказания на тесте.

### Предсказание на тестовой выборке

In [89]:
# Считаем TF-IDF на тренировочной выборке
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit_transform(corpus)

In [90]:
# Считаем TF-IDF на тестовой выборке
tf_idf_test = count_tf_idf.transform(corpus_test)

In [98]:
pred = randomized_search_lgbm.best_estimator_.named_steps['model'].predict(
    tf_idf_test)

In [99]:
print(f'Метрика F1 на тесте: {f1_score(y_test, pred):.2f}')

Метрика F1 на тесте: 0.77


На тестовой выборке тоже получаем результат 0.77. 

## *Дополнительно BERT*

Попробуем улучшить результат и воспользуемся моделью BERT. Файлы, config, vocab и model взяты c этого сайта [huggingface](https://huggingface.co/google-bert/bert-large-cased/tree/main).

Так как я ограничен технически то полным датафреймом воспользоваться не получиться (в колаб кончается память, в кагл очень долго считает (~145ч), на моем компьютере тоже самое, что и на кагл). В начале изучения данных увидели, что у нас есть дисбаланс классов, поэтому для BERT воспользуемся андерсамлингом и уменьшим наш мажорный класс. (Использую BERT только для выборки из 500 наблюдений)

In [ ]:
# В репозитории BERT просят оставить ссылку на статью
@article{turc2019,
         title = {Well-Read Students Learn Better: On the Importance of Pre-training Compact Models},
         author = {Turc, Iulia and Chang, Ming -
                   Wei and Lee, Kenton and Toutanova, Kristina},
         journal = {arXiv preprint arXiv: 1908.08962v2},
         year = {2019}
         }

In [6]:
# Отделяем таргет от остальных значений
X = df_toxic.drop(columns='toxic', axis=1)
y = df_toxic['toxic']

In [7]:
# Делаем андерсэмплинг
sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(X, y)

In [8]:
# Создаем сэмлированый датафрейм
df_resample = pd.concat([X_resample, y_resample], axis=1)

In [9]:
# Проверяем баланс классов
df_resample['toxic'].value_counts()

toxic
0    16186
1    16186
Name: count, dtype: int64

In [34]:
# Выбираем случайные 500 значений
df_small = df_resample.sample(500)

In [35]:
# Инициализируем токенизатор
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-cased')

In [37]:
# Преобразуем текст в номера токенов из словаря
tokenized = df_small['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

Token indices sequence length is longer than the specified maximum sequence length for this model (884 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (723 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1237 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (893 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

In [38]:
# Ищем максимальную последовательность
max_len = max(len(token) for token in tokenized.values)
print(max_len)

2840


Так как данная модель BERT имеет максимальную возможную длину вектора 512, то проверим, сколько значений превышают этот порог

In [39]:
# Создаем счетчик и пустой список для индексов
cnt = 0
idx = []

# Считаем количество значений, которые превышают порог
for i in tokenized.index:
    if len(tokenized[i]) > 512:
        cnt += 1
        idx.append(i)
print(f'Количество значений, превышающих максимальную длину: {cnt}')

Количество значений, превышающих максимальную длину: 15


Таких значений мало, а значит их можно спокойно удалить

In [40]:
# Удаляем значения превышающие порог
tokenized = tokenized.drop(index=idx)

In [41]:
# Ищем максимальную последовательность
max_len = max(len(token) for token in tokenized.values)
print(max_len)

501


In [42]:
# Применим метод padding
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [43]:
# Проверим размерность
padded.shape

(485, 501)

In [44]:
# Создаем маску "внимания"
attention_mask = np.where(padded !=0, 1, 0)

In [45]:
# Инициалзируем модель
model = transformers.BertModel.from_pretrained('bert-large-cased')

In [46]:
device = torch.device(
    "cuda:0") if torch.cuda.is_available() else torch.device("cpu")

# Указываем размер батча и создаем пустой список для эмбеддингов
batch_size = 97
embeddings = []
model.to(device)   # Закидываем модель на GPU

# Создаем цикл по батчу
for i in tqdm(range(padded.shape[0] // batch_size)):
    start_index = batch_size*i
    end_index = batch_size*(i+1)
    # Преобразуем данные
    batch = torch.LongTensor(padded[start_index:end_index]).to(
        device)  # Закидываем тензор на GPU
    # Преобразуем маску
    attention_mask_batch = torch.LongTensor(
        attention_mask[start_index:end_index]).to(device)

    # Указываем, что не используем градиент
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    # Переводим обратно на проц
    embeddings.append(batch_embeddings[0][:, 0, :].cpu().numpy())
    del batch
    del attention_mask_batch
    del batch_embeddings

  0%|          | 0/5 [00:00<?, ?it/s]

In [47]:
# Собираем все эмбеддинги в матрицу признаков
features = np.concatenate(embeddings)

In [48]:
# Создаем датафрейм эмбеддингов
df_embeddings = pd.DataFrame(features)

In [49]:
# Удаляем значения из датафрейма, которые превышают порог
df_small_idx = df_small.drop(index=idx)

In [50]:
# Обновляем значения индексов
df_embeddings.index = df_small_idx.index

In [51]:
# Добавляем в датафрейм эмбеддингов столбец с тагретом
df_embeddings['toxic'] = df_small_idx['toxic']

In [52]:
# Смотрим на первые 5 строк
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,toxic
90720,-0.605833,-0.386955,0.894628,0.504852,-0.828088,-0.106698,-0.003293,0.527482,0.001371,0.075519,...,0.595570,0.046099,0.001099,0.607826,0.128096,-0.486376,-0.869532,-0.360505,0.193627,0
3311,-0.266143,-0.793055,0.743506,0.167285,-0.050390,0.164040,-0.566893,0.703971,0.279710,-0.486641,...,0.382834,0.165652,-0.195410,0.529792,-0.417769,-0.143766,-0.373297,-0.262777,0.268213,0
152898,0.130858,-0.467700,0.747462,-0.181684,-0.320892,-0.248948,-0.697752,0.517300,0.028104,-0.493354,...,0.639658,0.324668,0.018745,0.514426,-0.321434,-0.526317,-0.402321,0.079822,0.409070,1
42690,-0.485343,-0.276089,0.306762,0.071361,-0.347920,-0.272762,-0.622653,0.622671,0.111816,-0.349448,...,0.213279,0.068472,0.332071,0.646844,0.054494,0.149767,-0.384126,-0.389681,-0.138949,0
91151,0.141174,-0.370319,0.094330,0.325538,0.091724,-0.326363,-0.480000,0.342367,-0.090163,-0.172868,...,0.396844,-0.236613,0.182056,0.764871,-0.450307,-0.195933,-0.080253,-0.577175,0.126797,1


In [53]:
# Отделяем таргет от остальных значений
X = df_embeddings.drop(columns='toxic', axis=1)
y = df_embeddings['toxic']

In [54]:
# Разделяем данные на тренировочную и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=RANDOM_STATE,
    test_size=TEST_SIZE,
    stratify=y
)

Будем использовать лучшую модель из предыдущего пунтка, а именно `LGBMClassifier` c параметрами max_depth=7, min_child_samples=30, n_estimators=2000, num_leaves=100

In [55]:
# Инициализируем модель
lgbm = LGBMClassifier(random_state=RANDOM_STATE, n_jobs=-1, objective='binary',
                      verbosity=-1, max_depth=7, min_child_samples=30, n_estimators=2000, num_leaves=100)

In [56]:
# Обучаем модель
lgbm.fit(X_train, y_train)

LGBMClassifier(max_depth=7, min_child_samples=30, n_estimators=2000, n_jobs=-1,
               num_leaves=100, objective='binary', random_state=42,
               verbosity=-1)

In [57]:
# Делаем предсказания на тесте
preds = lgbm.predict(X_test)

In [58]:
# Считаем метрику f1
print(f'Результат на тесте: {f1_score(y_test, preds):.2f}')

Результат на тесте: 0.78


По итогу получаем метрику 0.78. Скорее всего, если бы получилось использовать весь датафрейм результат был бы гораздо лучше.

## Выводы

Данные были получены из файла:

`toxic_comments.csv`
Сначала был сделан обзор данных. Само исследование проходило в 6 этапов:

1. Изучение и подготовка данных.
2. Обучение моделей
3. Вывод

**1. Изучение и подготовка данных.**

Сохранили файл в переменную `df_toxic`. Размерность файла - (159292, 2). Присутствует дисбаланс классов.

Данные подготовили следующим образом. Создали корпусов текстов и сделали лемматизацию, используя модуль `nltk`, то есть привели слова к начальной форме, после этого отчистили текст от лишних знаков, чтобы остались только буквы. Удалили стоп-слова. Разделили данные на тренировочные и тестовые. Оценку важности слов (`TF-IDF`) проводили в пайплайне в месте с обучением модели с использованием кросс-волидации.

**2. Обучение моделей**

Перед нами стояла задача классификации. Целевой признак `toxic`. Обучали 4 модели с использованием кросс-валидации:
- `LogisticRegression`
- `DecisionTreeClassifier`
- `SVM`
- `LGBMClassifier`

Проверяли результаты моделей на метрике `F1`, она должна была быть больше 0.75. После этого выбрали лучшую модель и подобрали ей гиперпараметры методом `RandomizedSearchCV`.

Сравнили все модели между собой и свели результат в таблицу

| **Модели**  |  **F1 на кросс-волидации** | **Отклонение метрики**  | **Удовлетворяет условию** |
|:------------- |:---------------:| :-------------:| :-------------:|
| **LogisticRegressionCV**  |<span style="color:red"> 0.72</span> | 0.57 | ❌ |
| **SVM** |<span style="color:red"> 0.74</span>| 0.53 | ❌ |
| **DecisionTreeRegressor**   |<span style="color:red"> 0.71</span>| 0.31 | ❌|
| **LGBMClassifier**   |<span style="color:red"> 0.74</span>| 0.85 | ❌|

По итогу был выбор между `SVM` и `LGBMClassifier`. Хоть отклонение у `LGBMClassifier` больше, но она обучалась быстрее, поэтому подбирали гиперпараметры для нее. Подбирали следующие гиперпараметры:

- `num_leaves`
- `n_estimators`
- `min_child_samples`
- `max_depth`

В результате получили модель с параметрами max_depth=7, min_child_samples=30, n_estimators=2000, num_leaves=100. Метрика F1 0.77. Так как метрика на кросс-валидации удовлетворяет условию. То выбрали данную модель как итоговую и сделали проверку на тесте. По итогу метрика на тесте составила 0.77.


***Дополнительный пункт BERT***

Данный пункт является дополнительным, так как из-за технических ограничений в полной мере воспользоваться `BERT` не представлялось возможным, поэтому `BERT` опробован на 500 наблюдений. Так как у нас наблюдается дисбаланс классов был сделан андерсэмплинг.

Преобразовали текст в номера токенов из словаря. Так как данная модель BERT имеет максимальную возможную длину вектора 512, то проверили, сколько значений превышают этот порог. Максимальная длина вектора у нас 2840 и 15 значений превысили данный порог. Эти 15 значений были удалены. Далее воспользовались методом padding и создали маску "внимания". Нашли эмбендинги для каждого наблюдения. И приступили к обучению модели. Взяли лучшую модель по итогам работы, а именно `LGBMClassifier`. В результате получили метрику `F1` на тесте 0.78.

**Вывод**

По итогу работы была сделана модель, которая может искать токсичные комментарии. Были обучены 4 модели и лучшая из них `LGBMClassifier` с метрикой `F1` 0.77, что удовлетворяет условию задачи (>0.75). Также была попытка воспользоваться `BERT`, но из-за технических ограничений в полную силу с ним поработать не получилось (на тесте - 0.78)

Данную работу можно улучшить, если использовать `BERT` на всем датафрейме или использовать специальную модель `unitary/toxic-bert`.